<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.3.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [10]:
# Download and prepare the horses or humans dataset

splits, info = tfds.load('horses_or_humans', as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling horses_or_humans-train.tfrecord...:   0%|          | 0/1027 [00:00<?, ? examples/s]

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling horses_or_humans-test.tfrecord...:   0%|          | 0/256 [00:00<?, ? examples/s]

Dataset horses_or_humans downloaded and prepared to C:\Users\habib\tensorflow_datasets\horses_or_humans\3.0.0. Subsequent calls will reuse this data.


In [11]:
splits

[<PrefetchDataset shapes: ((300, 300, 3), ()), types: (tf.uint8, tf.int64)>,
 <PrefetchDataset shapes: ((300, 300, 3), ()), types: (tf.uint8, tf.int64)>,
 <PrefetchDataset shapes: ((300, 300, 3), ()), types: (tf.uint8, tf.int64)>]

In [13]:
(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [15]:
print(num_examples)
print(num_classes)

1027
2


In [16]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [17]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [18]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [19]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [20]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [21]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [22]:
!rm -rf logs

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [23]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
 1/26 [>.............................] - ETA: 0s - loss: 0.7332 - accuracy: 0.4062WARNING:tensorflow:From C:\Users\habib\.conda\envs\tf\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


Instructions for updating:
use `tf.profiler.experimental.stop` instead.


26/26 [==============================] - 5s 187ms/step - loss: 0.6777 - accuracy: 0.5742 - val_loss: 0.6556 - val_accuracy: 0.7707
Epoch 2/10
26/26 [==============================] - 4s 167ms/step - loss: 0.6384 - accuracy: 0.6995 - val_loss: 0.6180 - val_accuracy: 0.7415
Epoch 3/10
26/26 [==============================] - 4s 168ms/step - loss: 0.5928 - accuracy: 0.6995 - val_loss: 0.5598 - val_accuracy: 0.8098
Epoch 4/10
26/26 [==============================] - 4s 171ms/step - loss: 0.5437 - accuracy: 0.7506 - val_loss: 0.5116 - val_accuracy: 0.7951
Epoch 5/10
26/26 [==============================] - 4s 170ms/step - loss: 0.5115 - accuracy: 0.7701 - val_loss: 0.4916 - val_accuracy: 0.7610
Epoch 6/10
26/26 [==============================] - 5s 179ms/step - loss: 0.4557 - accuracy: 0.8114 - val_loss: 0.3980 - val_accuracy: 0.8488
Epoch 7/10
26/26 [==============================] - 5s 193ms/step - loss: 0.4045 - accuracy: 0.8309 - val_loss: 0.3447 - val_accuracy: 0.8585
Epoch 8/10
26/26 

In [25]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 14948), started 0:21:20 ago. (Use '!kill 14948' to kill it.)

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [26]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.63.h5
26/26 - 5s - loss: 0.6748 - accuracy: 0.5560 - val_loss: 0.6338 - val_accuracy: 0.7171
Epoch 2/5

Epoch 00002: saving model to weights.02-0.69.h5
26/26 - 4s - loss: 0.6103 - accuracy: 0.7032 - val_loss: 0.6885 - val_accuracy: 0.5659
Epoch 3/5

Epoch 00003: saving model to weights.03-0.51.h5
26/26 - 4s - loss: 0.5630 - accuracy: 0.7482 - val_loss: 0.5119 - val_accuracy: 0.7854
Epoch 4/5

Epoch 00004: saving model to weights.04-0.56.h5
26/26 - 4s - loss: 0.5323 - accuracy: 0.7640 - val_loss: 0.5633 - val_accuracy: 0.6195
Epoch 5/5

Epoch 00005: saving model to weights.05-0.38.h5
26/26 - 4s - loss: 0.4564 - accuracy: 0.8187 - val_loss: 0.3809 - val_accuracy: 0.8976


In [27]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: saved_model\assets


INFO:tensorflow:Assets written to: saved_model\assets


26/26 - 7s - loss: 0.6828 - accuracy: 0.5438 - val_loss: 0.6837 - val_accuracy: 0.4439


In [28]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 4s - loss: 0.6751 - accuracy: 0.5596 - val_loss: 0.6588 - val_accuracy: 0.7366
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 4s - loss: 0.6455 - accuracy: 0.6849 - val_loss: 0.6335 - val_accuracy: 0.7171


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [29]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 4s - loss: 0.6733 - accuracy: 0.5973 - val_loss: 0.6634 - val_accuracy: 0.5220
Epoch 2/50
26/26 - 4s - loss: 0.6092 - accuracy: 0.6983 - val_loss: 0.5803 - val_accuracy: 0.7512
Epoch 3/50
26/26 - 4s - loss: 0.5758 - accuracy: 0.7238 - val_loss: 0.5262 - val_accuracy: 0.7512
Epoch 4/50
26/26 - 4s - loss: 0.4962 - accuracy: 0.7713 - val_loss: 0.4939 - val_accuracy: 0.7951
Epoch 5/50
26/26 - 4s - loss: 0.4383 - accuracy: 0.8017 - val_loss: 0.4000 - val_accuracy: 0.8244
Epoch 6/50
26/26 - 4s - loss: 0.4140 - accuracy: 0.8163 - val_loss: 0.3659 - val_accuracy: 0.8585
Epoch 7/50
26/26 - 4s - loss: 0.3688 - accuracy: 0.8540 - val_loss: 0.6042 - val_accuracy: 0.6780
Epoch 8/50
26/26 - 4s - loss: 0.3130 - accuracy: 0.8808 - val_loss: 0.2829 - val_accuracy: 0.8927
Epoch 9/50
26/26 - 4s - loss: 0.2302 - accuracy: 0.9355 - val_loss: 0.2192 - val_accuracy: 0.9171
Epoch 10/50
26/26 - 4s - loss: 0.2482 - accuracy: 0.9100 - val_loss: 0.2004 - val_accuracy: 0.9317
Epoch 11/50
26/26 -

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [30]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 4s 169ms/step - loss: 0.6826 - accuracy: 0.5608 - val_loss: 0.6555 - val_accuracy: 0.5415
Epoch 2/5
26/26 [==============================] - 4s 166ms/step - loss: 0.6112 - accuracy: 0.6436 - val_loss: 0.5733 - val_accuracy: 0.7610
Epoch 3/5
26/26 [==============================] - 4s 166ms/step - loss: 0.5744 - accuracy: 0.7372 - val_loss: 0.5430 - val_accuracy: 0.7951
Epoch 4/5
26/26 [==============================] - 4s 165ms/step - loss: 0.5047 - accuracy: 0.7859 - val_loss: 0.5838 - val_accuracy: 0.6488
Epoch 5/5
26/26 [==============================] - 4s 165ms/step - loss: 0.4392 - accuracy: 0.8029 - val_loss: 0.3819 - val_accuracy: 0.8585


In [31]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.560827,0.682628,0.541463,0.655507
1,1,0.643552,0.611193,0.760976,0.573341
2,2,0.737226,0.574376,0.795122,0.543030
3,3,0.785888,0.504661,0.648780,0.583756
4,4,0.802920,0.439235,0.858537,0.381919


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [32]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
    initial_lr = 0.01
    drop = 0.5
    epochs_drop = 1
    lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
 2/26 [=>............................] - ETA: 33s - loss: 0.7018 - accuracy: 0.3906WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1553s vs `on_train_batch_end` time: 2.6533s). Check your callbacks.


26/26 [==============================] - 7s 277ms/step - loss: 0.6812 - accuracy: 0.5255 - val_loss: 0.6882 - val_accuracy: 0.4341

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 4s 167ms/step - loss: 0.6617 - accuracy: 0.5511 - val_loss: 0.6709 - val_accuracy: 0.4927

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 4s 168ms/step - loss: 0.6521 - accuracy: 0.5876 - val_loss: 0.6653 - val_accuracy: 0.5268

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 4s 167ms/step - loss: 0.6469 - accuracy: 0.5949 - val_loss: 0.6626 - val_accuracy: 0.5366

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 4s 166ms/step - loss: 0.6443 - accuracy: 0.6180 - val_loss: 0.6616 - val_accuracy: 0.5366


In [33]:
%tensorboard --logdir log_dir

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 22164.

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

In [ ]:
%tensorboard --logdir log_dir